In [26]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

import matplotlib.pyplot as plt
from matplotlib import rcParams
import datetime as dt

#Data Collection#
stocks = ['RBSPF']

start = dt.datetime(2010, 1, 1)
end = dt.datetime(2030, 1, 27)

data = web.DataReader('RBSPF', 'yahoo', start, end)

#Data pre-processing#

dates =[]
for x in range(len(data)):
    '''Remove the level index caused by the multiple layers in the original dataset'''
    newdate = str(data.index[x])
    newdate = newdate[0:10]
    dates.append(newdate)

data['Dates'] = dates

#nblevels = ticker.index.nlevels
#print(nblevels)
#ticker.reset_index()
#pd.MultiIndex.droplevel(level=1)
#ticker.droplevel('Date')
#ticker.droplevel('Date', axis=1)
#ticker.reset_index(level=1, drop=True)
data.reset_index(drop=True, inplace=True)
#ticker["DatesNew"]= pd.to_datetime(ticker["Dates"])
#ticker.columns = ticker.columns.droplevel()
#display(data)

#ticker = ticker['Adj Close']
#display(ticker)
#ticker.rename({'Date':'Date', '':'Adj Close'}, axis=1)
#display(ticker)
del data['Dates']
#display(data)

#Scaling the features
scaler = MinMaxScaler(feature_range=(0,1))
#Transform the data into
scaled_Data = scaler.fit_transform(data)
data['Dates'] = dates
df = pd.DataFrame(data=scaled_Data, index=[data.Dates], columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
df['Dates'] = dates
df.reset_index(drop=True, inplace=True)
display(df)

df = df[['Dates','Adj Close']]

display(df)

#Create a separate dataframe for Adj Close data and convert into an array
#data = data.filter['Adj Close']
#Setting the index
df.index = df.Dates
df.drop('Dates', axis=1, inplace=True)
#Creating train and test sets
dataset = df['Adj Close'].values
display(dataset)
#Train/Test split
train = dataset[0:2196]
valid = dataset[2196:]

#_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
dataset
#Converting dataset into x_train and y_train
x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_Data[i-60:i,0])
    y_train.append(scaled_Data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

#Create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

#predicting 246 values, using past 60 from the train data
data['Dates'] = pd.to_datetime(data["Dates"])
data['Dates'] = (data['Dates'] - data['Dates'].min()) / np.timedelta64(1,'D')
inputs = data[len(data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = scaler.transform(inputs)

##df = df.drop(df.columns[[0, 1, 3]], axis=1)##

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
print(rms)

#Plotting
train = data[:2196]
valid = data[2196:]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])





,Open,High,Low,Close,Adj Close,Volume,Dates
0,0.046589,0.038462,0.038397,0.038333,0.001094,0.038333,2010-01-04
1,0.048253,0.046823,0.046745,0.048333,0.001094,0.048333,2010-01-05
2,0.054908,0.055184,0.055092,0.055000,0.000209,0.055000,2010-01-06
3,0.054908,0.048495,0.055092,0.055000,0.002083,0.055000,2010-01-07
4,0.048253,0.048495,0.048414,0.048333,0.000590,0.048333,2010-01-08
...,...,...,...,...,...,...,...
2751,0.337770,0.322742,0.338898,0.321667,0.000371,0.376154,2020-12-07
2752,0.321131,0.322742,0.322204,0.321667,0.000000,0.376154,2020-12-08
2753,0.331115,0.322742,0.332220,0.330000,0.000171,0.385732,2020-12-09
2754,0.314476,0.316054,0.315526,0.315000,0.000190,0.368492,2020-12-10


,Dates,Adj Close
0,2010-01-04,0.001094
1,2010-01-05,0.001094
2,2010-01-06,0.000209
3,2010-01-07,0.002083
4,2010-01-08,0.000590
...,...,...
2751,2020-12-07,0.000371
2752,2020-12-08,0.000000
2753,2020-12-09,0.000171
2754,2020-12-10,0.000190


array([0.00109398, 0.00109398, 0.00020928, ..., 0.00017123, 0.00019026,
       0.        ])

Epoch 1/1
 - 708s - loss: 0.0028


ValueError: non-broadcastable output operand with shape (4340,1) doesn't match the broadcast shape (4340,6)